# Data

Import the main packages used for handling files and datasets.

In [2]:
using BenchmarkTools 
using DataFrames 
using DelimitedFiles 
using CSV
using XLSX

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1278
┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1278
┌ Info: Precompiling XLSX [fdbf4ff8-1666-58a4-91e7-1b58723a45e0]
└ @ Base loading.jl:1278


## 1. Download files

Download a CSV file. Julia provides the *download* function, which uses the the commands *wget*, *curl* or *fetch*. 

In [19]:
P = download("https://raw.githubusercontent.com/nassarhuda/easy_data/master/programming_languages.csv",
    "data/programminglanguages.csv")

"data/programminglanguages.csv"

As we can see, the dowloaded file is a simple CSV file with two columns (year, language).

In [20]:
;head "data/programminglanguages.csv"

year,language
1951,Regional Assembly Language
1952,Autocode
1954,IPL
1955,FLOW-MATIC
1957,FORTRAN
1957,COMTRAN
1958,LISP
1958,ALGOL 58
1959,FACT


## 2. Read (and write) files

### DelimitedFiles package

There are two main ways of reading files in Julia. The first option consists in using the *DelimitedFiles* package, which provides the fuction **readdlm**. However, this package should only be used when the file to be read is really complicated - not the case -.

In [21]:
B, H = readdlm(P, ','; header=true)

(Any[1951 "Regional Assembly Language"; 1952 "Autocode"; … ; 2012 "Julia"; 2014 "Swift"], AbstractString["year" "language"])

The file's path (P) and the delimeter (',') have been provided. The *header* option tells the function to return the data's body (B) and header (H) separately.

In [22]:
H #header

1×2 Array{AbstractString,2}:
 "year"  "language"

In [23]:
B[1:5, :] #first 5 elements of the body

5×2 Array{Any,2}:
 1951  "Regional Assembly Language"
 1952  "Autocode"
 1954  "IPL"
 1955  "FLOW-MATIC"
 1957  "FORTRAN"

The *DelimetedFiles* package also provides a method to write a text file.

In [24]:
writedlm("data/programminglanguages_dlm.txt", P, '-')

### CSV package

Using the *CSV* package has the following advantages over the *DelimitedFiles* package: 

- Directly converts the data into a DataFrame. 
- It's faster

In [25]:
C = CSV.read(P);

In [26]:
typeof(C) # 

DataFrame

In [27]:
C[1:5, :]

,year,language
,Int64,String
1,1951,Regional Assembly Language
2,1952,Autocode
3,1954,IPL
4,1955,FLOW-MATIC
5,1957,FORTRAN


DataFrames allow different ways of accessing the different files.

In [28]:
C[1:5, :year] #by key

5-element Array{Int64,1}:
 1951
 1952
 1954
 1955
 1957

In [29]:
C.year[1:5] #by dot notation

5-element Array{Int64,1}:
 1951
 1952
 1954
 1955
 1957

In order to see which fields the DataFrame contains, we can use the *names* function.

In [30]:
names(C)

2-element Array{String,1}:
 "year"
 "language"

We can check that the *CSV* package is faster than the *DelimitedFiles* package with the *@btime* macro.

In [31]:
@btime B, H  = readdlm(P, ','; header=true);
@btime C = CSV.read(P);

  64.316 μs (326 allocations: 51.11 KiB)
  48.089 μs (173 allocations: 18.89 KiB)


In order to write to a *.csv file using the CSV package

In [32]:
CSV.write("data/programminglanguages_csv.csv", DataFrame(B))

"data/programminglanguages_csv.csv"

### XLSX package

The *XLSX* package allows to read *.xlxs files, specifying the sheet and the cell range to be read with the **readdata** function.

In [33]:
cellRange = XLSX.readdata("data/zillow_data_download_april2020.xlsx", # file name
    "Sale_counts_city", # sheet name
    "A1:F9" # cell range
    )

9×6 Array{Any,2}:
      "RegionID"  "RegionName"    …      "2008-03"      "2008-04"
  6181            "New York"             missing        missing
 12447            "Los Angeles"      1446           1705
 39051            "Houston"          2926           3121
 17426            "Chicago"          2910           3022
  6915            "San Antonio"   …  1479           1529
 13271            "Philadelphia"     1609           1795
 40326            "Phoenix"          1310           1519
 18959            "Las Vegas"        1618           1856

In case we don't want to specify any cell range, we can read the whole sheet using the **readtable** function.

In [34]:
wholeSheet = XLSX.readtable("data/zillow_data_download_april2020.xlsx", "Sale_counts_city")

(Any[Any[6181, 12447, 39051, 17426, 6915, 13271, 40326, 18959, 54296, 38128  …  396952, 397236, 398030, 398104, 398357, 398712, 398716, 399081, 737789, 760882], Any["New York", "Los Angeles", "Houston", "Chicago", "San Antonio", "Philadelphia", "Phoenix", "Las Vegas", "San Diego", "Dallas"  …  "Barnard Plantation", "Windsor Place", "Stockbridge", "Mattamiscontis", "Chase Stream", "Bowdoin College Grant West", "Summerset", "Long Pond", "Hideout", "Ebeemee"], Any["New York", "California", "Texas", "Illinois", "Texas", "Pennsylvania", "Arizona", "Nevada", "California", "Texas"  …  "Maine", "Missouri", "Wisconsin", "Maine", "Maine", "Maine", "South Dakota", "Maine", "Utah", "Maine"], Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  28750, 28751, 28752, 28753, 28754, 28755, 28756, 28757, 28758, 28759], Any[missing, 1446, 2926, 2910, 1479, 1609, 1310, 1618, 772, 1158  …  0, 0, 0, 0, 0, 0, 0, 0, 1, 0], Any[missing, 1705, 3121, 3022, 1529, 1795, 1519, 1856, 1057, 1232  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], A

In [35]:
typeof(wholeSheet)

Tuple{Array{Any,1},Array{Symbol,1}}

The returned value is a tuple of 2 items. The first item is a vector of vectors, with each vector containing a column of the sheet. The second item contains the header.

In [36]:
wholeSheet[2][1:4]

4-element Array{Symbol,1}:
 :RegionID
 :RegionName
 :StateName
 :SizeRank

In [37]:
wholeSheet[1][3][1:4] # :StateName column 

4-element Array{Any,1}:
 "New York"
 "California"
 "Texas"
 "Illinois"

We can store the data as a *DataFrame* as follows:

In [38]:
wholeSheetDF = DataFrame(wholeSheet...); # equivalent to DataFrame(wholeSheet[1], wholeSheet[2])

In [39]:
wholeSheetDF[1:5, :]

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06
,Any,Any,Any,Any,Any,Any,Any,Any
1,6181,New York,New York,1,missing,missing,missing,missing
2,12447,Los Angeles,California,2,1446,1705,1795,1890
3,39051,Houston,Texas,3,2926,3121,3220,3405
4,17426,Chicago,Illinois,4,2910,3022,2937,3224
5,6915,San Antonio,Texas,5,1479,1529,1582,1761


In [40]:
wholeSheetDF.StateName[1:4] # StateName column

4-element Array{Any,1}:
 "New York"
 "California"
 "Texas"
 "Illinois"

We can also easily write data to an XLSX file

In [41]:
XLSX.writetable("data/writable_using_XLSX.xlsx", wholeSheet[1], wholeSheet[2])

### DataFrames

We can create a DataFrame from scratch as follows:

In [42]:
countries = ["Spain", "Italy", "UK"]
meals = ["paella", "pasta", "fish and chips"]
popullations = [47, 55, 64]

dfMeals = DataFrame(item=countries, meal=meals)
dfPopullations = DataFrame(item=countries, popullation=popullations)

,item,popullation
,String,Int64
1,Spain,47
2,Italy,55
3,UK,64


In [43]:
dfAll = innerjoin(dfMeals, dfPopullations, on=:item)

,item,meal,popullation
,String,String,Int64
1,Spain,paella,47
2,Italy,pasta,55
3,UK,fish and chips,64


A DataFrame can also be built from a matrix

In [44]:
M = ["Spain" "paella" 47; 
     "Italy" "pasta" 55;
     "UK"  "fish and chips" 60]

dfMatrix = DataFrame(M)

,x1,x2,x3
,Any,Any,Any
1,Spain,paella,47
2,Italy,pasta,55
3,UK,fish and chips,60


## Importing data

Our data might be in other formats different from *txt*, *csv* or *xlsx*. The  **JLD**, **NPZ**, **RData** and **MAT** libraries allow to import jld, npz, rda and mat files.

In [71]:
using JLD
using NPZ 
using RData 
using MAT

In [63]:
jldData  = JLD.load("data/mytempdata.jld") # JLD.load
println("typeof(jldData) = $(typeof(jldData))\n")
printstyled(jldData["tempdata"])
JLD.save("data/mywrite.jld", "A", jldData) # JLD.save

typeof(jldData) = Dict{String,Any}

[2 1446 1705 1795 1890; 3 2926 3121 3220 3405; 4 2910 3022 2937 3224; 5 1479 1529 1582 1761]

In [68]:
npzData = NPZ.npzread("data/mytempdata.npz") # NPZ.npzread
println("typeof(npzData) = $(typeof(npzData))\n")
printstyled(npzData)
NPZ.npzwrite("data/mywrite.npz", npzData) # NPZ.npzwrite

typeof(npzData) = Array{Int64,2}

[2 1446 1705 1795 1890; 3 2926 3121 3220 3405; 4 2910 3022 2937 3224; 5 1479 1529 1582 1761]

In [74]:
rData = RData.load("data/mytempdata.rda") # RData.load 
println("typeof(rData) = $(typeof(rData))\n")
printstyled(rData["tempdata"])
# We'll need RCall to save here. https://github.com/JuliaData/RData.jl/issues/56
#using RCall
#@rput R_data
#R"save(R_data, file=\"mywrite.rda\")"

typeof(rData) = Dict{String,Any}

Int32[2 1446 1705 1795 1890; 3 2926 3121 3220 3405; 4 2910 3022 2937 3224; 5 1479 1529 1582 1761]

In [78]:
matlabData = MAT.matread("data/mytempdata.mat") # MAT.matread 
println("typeof(matlabData) = $(typeof(matlabData))\n")
printstyled(matlabData["tempdata"])
MAT.matwrite("data/mywrite.mat", matlabData) # MAT.matwrite

typeof(matlabData) = Dict{String,Any}

[2 1446 1705 1795 1890; 3 2926 3121 3220 3405; 4 2910 3022 2937 3224; 5 1479 1529 1582 1761]

## 3. Data processing

In [80]:
B

73×2 Array{Any,2}:
 1951  "Regional Assembly Language"
 1952  "Autocode"
 1954  "IPL"
 1955  "FLOW-MATIC"
 1957  "FORTRAN"
 1957  "COMTRAN"
 1958  "LISP"
 1958  "ALGOL 58"
 1959  "FACT"
 1959  "COBOL"
 1959  "RPG"
 1962  "APL"
 1962  "Simula"
    ⋮  
 2003  "Scala"
 2005  "F#"
 2006  "PowerShell"
 2007  "Clojure"
 2009  "Go"
 2010  "Rust"
 2011  "Dart"
 2011  "Kotlin"
 2011  "Red"
 2011  "Elixir"
 2012  "Julia"
 2014  "Swift"

 **Question 1: Which year was a given language invented ?**

In [91]:
function year_invented(dataset, lang::AbstractString)
    loc = findfirst(isequal(lang), dataset[:,2])
    !isnothing(loc) && return dataset[loc,1]
    error("Error: language '$(lang)' not found.")
end

yearInvented (generic function with 1 method)

In [92]:
year_invented(B, "Julia")

2012

In [96]:
yearInvented(B, "Pp")

ErrorException: Error: language 'Pp' not found.

**Question 2: How many languages were created in a given year?**

In [110]:
function languages_created(dataset, year::Integer)
    count(x -> x == year, dataset[:,1])
end

languagesCreated (generic function with 1 method)

In [114]:
languages_created(B, 2011)

4

#### Save the dataset into a dictionary

In [158]:
years = Set(B[:,1])
dataDict = Dict{Integer, Vector{String}}()

for year in years 
    indexes = findall(x -> x == year, B[:,1])
    dataDict[year] = B[indexes, 2]
end

In [162]:
dataDict

Dict{Integer,Array{String,1}} with 45 entries:
  1991 => ["Python", "Visual Basic"]
  1993 => ["Lua", "R"]
  2005 => ["F#"]
  2010 => ["Rust"]
  1983 => ["Ada"]
  1957 => ["FORTRAN", "COMTRAN"]
  1987 => ["Perl"]
  2007 => ["Clojure"]
  1989 => ["FL "]
  1969 => ["B"]
  1952 => ["Autocode"]
  1963 => ["CPL"]
  2003 => ["Groovy", "Scala"]
  1958 => ["LISP", "ALGOL 58"]
  2014 => ["Swift"]
  1951 => ["Regional Assembly Language"]
  1997 => ["Rebol"]
  2000 => ["ActionScript"]
  1967 => ["BCPL"]
  1985 => ["Eiffel"]
  1968 => ["Logo"]
  1955 => ["FLOW-MATIC"]
  1984 => ["Common Lisp", "MATLAB", "dBase III"]
  2009 => ["Go"]
  1966 => ["JOSS"]
  ⋮    => ⋮